
# 📘 Fundamentos de SQL no Databricks

Este notebook traz uma introdução prática aos principais comandos SQL, com foco em quem está começando. A ideia aqui é entender o papel de cada cláusula e saber como usá-las no dia a dia, consultando dados de forma eficiente e clara dentro do Databricks.

✅ O que você vai aprender:

- Como selecionar colunas e linhas com SELECT e WHERE
- Como agrupar dados com GROUP BY e filtrar agregações com HAVING
- Como ordenar resultados com ORDER BY
- Como unir e combinar tabelas usando JOINS e UNION

Todos os exemplos são simples, objetivos e pensados para facilitar o aprendizado de SQL no contexto do Spark SQL.

Links úteis:

- SQL language reference: https://docs.databricks.com/aws/en/sql/language-manual/

_____________________________________________________________

# SELECT

A cláusula SELECT no SQL é usada para consultar dados de uma ou mais tabelas. Ela define quais colunas você quer visualizar.

Quando estiver começando, preste atenção em três coisas:

- Ordem importa: primeiro SELECT, depois FROM, depois filtros (WHERE, GROUP BY, etc.).
- Evite SELECT *: sempre que possível, selecione só as colunas que precisa.
- Alias ajudam: use AS para renomear colunas e deixar o resultado mais legível.

In [0]:
-- Seleciona todas as colunas da tabela de pokémons
SELECT * FROM workspace.pokemons.pokemon

In [0]:
-- Seleciona colunas específicas da tabela de pokémons
SELECT 
  `#`, 
  Name, 
  `Type 1`, 
  `Type 2`, 
  HP, 
  Generation, 
  Legendary 
FROM workspace.pokemons.pokemon

In [0]:
-- Seleciona e renomeia colunas específicas da tabela de pokémons
SELECT 
  `#` AS pokedex_number, 
  Name AS pkmn_name, 
  `Type 1` AS pkmn_type_primary, 
  `Type 2` AS pkmn_type_secondary, 
  HP AS pkmn_hp, 
  Generation AS pkmn_generation, 
  Legendary AS pkmn_legendary
FROM workspace.pokemons.pokemon

In [0]:
-- Seleciona e renomeia colunas específicas da tabela de pokémons
SELECT 
  pkmn.`#` AS pokedex_number, 
  pkmn.Name AS pkmn_name, 
  pkmn.`Type 1` AS pkmn_type_primary, 
  pkmn.`Type 2` AS pkmn_type_secondary, 
  pkmn.HP AS pkmn_hp, 
  pkmn.Generation AS pkmn_generation, 
  pkmn.Legendary AS pkmn_legendary
FROM workspace.pokemons.pokemon pkmn

In [0]:
-- Seleciona todos os tipos primários distintos dos pokémons
SELECT DISTINCT `Type 1` FROM workspace.pokemons.pokemon

In [0]:
-- Retorna o tipo primário dos pokémons traduzido para português, agrupando em 'Grama', 'Fogo', 'Água' ou 'outros tipos'
SELECT
  DISTINCT  
  CASE 
    WHEN pkmn.`Type 1` = 'Grass' THEN 'Grama'
    WHEN pkmn.`Type 1` = 'Fire' THEN 'Fogo'
    WHEN pkmn.`Type 1` = 'Water' THEN 'Água'
    ELSE 'outros tipos' 
  END AS pkmn_type
FROM workspace.pokemons.pokemon pkmn

# WHERE 

A cláusula WHERE serve para filtrar os dados retornados pela consulta. Ela define quais linhas da tabela devem aparecer no resultado.

Quando estiver começando, fique atento a:

- Condições lógicas: use =, >, <, <=, >=, <> para comparar valores.
- Texto precisa de aspas: strings devem estar entre aspas simples ('João').
- Combine filtros com AND, OR e NOT.

In [0]:
-- Seleciona todos os pokémons cujo tipo primário é 'Grass'
SELECT * FROM workspace.pokemons.pokemon
WHERE `Type 1` = 'Grass'

In [0]:
-- Seleciona todos os pokémons cujo tipo primário não é 'Grass'
SELECT * FROM workspace.pokemons.pokemon
WHERE `Type 1` <> 'Grass'

In [0]:
-- Seleciona todos os pokémons com HP maior que 100
SELECT * FROM workspace.pokemons.pokemon
WHERE HP > 100

In [0]:
-- Seleciona todos os pokémons com HP menor ou igual a 100
SELECT * FROM workspace.pokemons.pokemon
WHERE HP <= 100

In [0]:
-- Seleciona todos os pokémons cujo nome começa com a letra 'P'
SELECT * FROM workspace.pokemons.pokemon
WHERE Name LIKE 'P%'

In [0]:
-- Seleciona todos os pokémons cujo nome termina com a letra 'a'
SELECT * FROM workspace.pokemons.pokemon
WHERE Name LIKE '%a'

In [0]:
-- Seleciona todos os pokémons cujo nome contém a letra 'k'
SELECT * FROM workspace.pokemons.pokemon
WHERE Name LIKE '%k%'

In [0]:
-- Seleciona todos os pokémons cujo 'Type 1' é 'Grass', 'Fire' ou 'Water'
SELECT * FROM workspace.pokemons.pokemon
WHERE `Type 1` IN ('Grass','Fire','Water')

In [0]:
-- Seleciona todos os pokémons cujo HP está entre 190 e 250
SELECT * FROM workspace.pokemons.pokemon
WHERE HP BETWEEN 190 AND 250

In [0]:
-- Seleciona todos os pokémons cujo tipo primário é 'Grass' e tipo secundário é 'Psychic'
SELECT * FROM workspace.pokemons.pokemon
WHERE `Type 1` = 'Grass' AND `Type 2` = 'Psychic'

In [0]:
-- Seleciona todos os pokémons cujo tipo primário é 'Grass' ou tipo secundário é 'Psychic'
SELECT * FROM workspace.pokemons.pokemon
WHERE `Type 1` = 'Grass' OR `Type 2` = 'Psychic'

In [0]:
-- Seleciona todos os pokémons cujo tipo primário é 'Grass' ou cujo tipo secundário não é 'Psychic'
SELECT * FROM workspace.pokemons.pokemon
WHERE `Type 1` = 'Grass' OR NOT `Type 2` = 'Psychic'

# GROUP BY

Usado para agrupar linhas com valores iguais em uma ou mais colunas. Geralmente vem junto com funções agregadas como COUNT(), SUM(), AVG(), etc.

`⚠️ Atenção: toda coluna no SELECT que não está dentro de uma função agregada precisa estar no GROUP BY.`

In [0]:
-- Conta a quantidade de pokémons por geração
SELECT
  Generation, 
  COUNT(1) AS qtd_pokemons
FROM workspace.pokemons.pokemon
GROUP BY 
  Generation

In [0]:
-- Conta a quantidade de pokémons por geração e combinação de tipos primário e secundário
SELECT
  Generation,
  `Type 1` AS pkmn_type_primary,
  `Type 2` AS pkmn_type_secondary, 
  COUNT(1) AS qtd_pokemons
FROM workspace.pokemons.pokemon
GROUP BY 
  Generation, `Type 1`, `Type 2`

# ORDER BY

Serve para ordenar os resultados da consulta, de forma crescente (ASC) ou decrescente (DESC).

`⚠️ Pode ordenar por colunas que estão no SELECT, por posição (ex: ORDER BY 1), ou por alias.`

In [0]:
-- Agrupa e conta a quantidade de pokémons por geração e tipos primário e secundário, ordenando pelo total de pokémons
SELECT
  Generation,
  `Type 1` AS pkmn_type_primary,
  `Type 2` AS pkmn_type_secondary, 
  COUNT(1) AS qtd_pokemons
FROM workspace.pokemons.pokemon
GROUP BY 
  Generation, `Type 1`, `Type 2`
ORDER BY 
  qtd_pokemons

In [0]:
-- Agrupa e conta a quantidade de pokémons por geração e tipos primário e secundário, ordenando pelo total de pokémons em ordem crescente
SELECT
  Generation,
  `Type 1` AS pkmn_type_primary,
  `Type 2` AS pkmn_type_secondary, 
  COUNT(1) AS qtd_pokemons
FROM workspace.pokemons.pokemon
GROUP BY 
  Generation, `Type 1`, `Type 2`
ORDER BY 
  qtd_pokemons

In [0]:
-- Agrupa e conta a quantidade de pokémons por geração e tipos primário e secundário, ordenando pelo total de pokémons e geração
SELECT
  Generation,
  `Type 1` AS pkmn_type_primary,
  `Type 2` AS pkmn_type_secondary, 
  COUNT(1) AS qtd_pokemons
FROM workspace.pokemons.pokemon
GROUP BY 
  Generation, `Type 1`, `Type 2`
ORDER BY 
  qtd_pokemons, Generation

In [0]:
-- Agrupa e conta a quantidade de pokémons por geração e tipos primário e secundário, ordenando pelo total de pokémons (decrescente) e geração (decrescente)
SELECT
  Generation,
  `Type 1` AS pkmn_type_primary,
  `Type 2` AS pkmn_type_secondary, 
  COUNT(1) AS qtd_pokemons
FROM workspace.pokemons.pokemon
GROUP BY 
  Generation, `Type 1`, `Type 2`
ORDER BY 
  qtd_pokemons DESC, Generation DESC

# JOIN

Permite combinar dados de duas ou mais tabelas com base em uma coluna em comum.

Tipos principais:

- **INNER JOIN:** retorna só os registros que batem nas duas tabelas
- **LEFT JOIN:** tudo da tabela da esquerda, mesmo sem correspondência na direita
- **RIGHT JOIN:** o contrário do LEFT
- **FULL JOIN:** tudo que está de ambos os lados
- **OUTER JOIN:** tudo que está de ambos os lados, quando houver correspondência

![1_Gz6WOE2SAYkpnFmWrV5tVQ.png](./img/1_Gz6WOE2SAYkpnFmWrV5tVQ.png "1_Gz6WOE2SAYkpnFmWrV5tVQ.png")

In [0]:
-- Faz um INNER JOIN entre as tabelas de pokémons e detalhes dos pokémons, trazendo informações combinadas de ambos
SELECT 
  pkmn.`#` AS pkmn_id,
  pkmn.Name AS pkmn_name,
  pkmn.`Type 1` AS pkmn_type_primary,
  pkmn.`Type 2` AS pkmn_type_secondary,
  pkmnd.Classification AS pkmn_classification,
  pkmnd.`Height (m)` AS pkmn_height,
  pkmnd.`Weight (kg)` AS pkmn_weight,
  pkmnd.Abilities AS pkmn_abilities
FROM 
  workspace.pokemons.pokemon pkmn
JOIN 
  workspace.pokemons.pokemon_details pkmnd ON pkmn.`#` = pkmnd.`Pokedex Number`

In [0]:
-- Retorna todos os registros da tabela de detalhes dos pokémons, junto com os dados correspondentes da tabela principal de pokémons (RIGHT JOIN)
SELECT 
  pkmn.`#` AS pkmn_id,
  pkmn.Name AS pkmn_name,
  pkmn.`Type 1` AS pkmn_type_primary,
  pkmn.`Type 2` AS pkmn_type_secondary,
  pkmnd.Classification AS pkmn_classification,
  pkmnd.`Height (m)` AS pkmn_height,
  pkmnd.`Weight (kg)` AS pkmn_weight,
  pkmnd.Abilities AS pkmn_abilities
FROM 
  workspace.pokemons.pokemon pkmn
RIGHT JOIN 
  workspace.pokemons.pokemon_details pkmnd ON pkmn.`#` = pkmnd.`Pokedex Number`

In [0]:
-- Retorna todos os registros da tabela principal de pokémons, junto com os dados correspondentes da tabela de detalhes dos pokémons (LEFT JOIN)
SELECT 
  pkmn.`#` AS pkmn_id,
  pkmn.Name AS pkmn_name,
  pkmn.`Type 1` AS pkmn_type_primary,
  pkmn.`Type 2` AS pkmn_type_secondary,
  pkmnd.Classification AS pkmn_classification,
  pkmnd.`Height (m)` AS pkmn_height,
  pkmnd.`Weight (kg)` AS pkmn_weight,
  pkmnd.Abilities AS pkmn_abilities
FROM 
  workspace.pokemons.pokemon pkmn
LEFT JOIN 
  workspace.pokemons.pokemon_details pkmnd ON pkmn.`#` = pkmnd.`Pokedex Number`

In [0]:
-- Retorna todos os registros das duas tabelas (pokemon e pokemon_details)
  pkmn.`#` AS pkmn_id,
  pkmn.Name AS pkmn_name,
  pkmn.`Type 1` AS pkmn_type_primary,
  pkmn.`Type 2` AS pkmn_type_secondary,
  pkmnd.Classification AS pkmn_classification,
  pkmnd.`Height (m)` AS pkmn_height,
  pkmnd.`Weight (kg)` AS pkmn_weight,
  pkmnd.Abilities AS pkmn_abilities
FROM 
  workspace.pokemons.pokemon pkmn
FULL JOIN 
  workspace.pokemons.pokemon_details pkmnd ON pkmn.`#` = pkmnd.`Pokedex Number`

In [0]:
-- Retorna todos os registros das duas tabelas (pokemon e pokemon_details), combinando onde houver correspondência (FULL OUTER JOIN)
SELECT 
  pkmn.`#` AS pkmn_id,
  pkmn.Name AS pkmn_name,
  pkmn.`Type 1` AS pkmn_type_primary,
  pkmn.`Type 2` AS pkmn_type_secondary,
  pkmnd.Classification AS pkmn_classification,
  pkmnd.`Height (m)` AS pkmn_height,
  pkmnd.`Weight (kg)` AS pkmn_weight,
  pkmnd.Abilities AS pkmn_abilities
FROM 
  workspace.pokemons.pokemon pkmn
OUTER JOIN 
  workspace.pokemons.pokemon_details pkmnd ON pkmn.`#` = pkmnd.`Pokedex Number`

# UNION

Serve para juntar os resultados de duas consultas, desde que tenham mesmo número de colunas e tipos compatíveis.

- UNION: elimina duplicados
- UNION ALL: mantém tudo, inclusive duplicados

In [0]:
-- Retorna todos os pokémons do tipo "Grass" e todos os pokémons classificados como "Bug" (usando UNION)
SELECT 
  pkmn.`#` AS pkmn_id,
  pkmn.Name AS pkmn_name,
  pkmn.Legendary AS pkmn_legendary
FROM 
  workspace.pokemons.pokemon pkmn
WHERE 
  pkmn.`Type 1` = "Grass"

UNION

SELECT 
  pkmnd.`Pokedex Number` AS pkmn_id,
  pkmnd.Name AS pkmn_name,
  pkmnd.`Legendary Status` AS pkmn_legendary
FROM 
  workspace.pokemons.pokemon_details pkmnd
WHERE 
  pkmnd.Classification LIKE '%Bug%'

In [0]:
-- Retorna todos os pokémons do tipo "Grass" e todos os pokémons classificados como "Bug" (usando UNION)
SELECT 
  pkmn.`#` AS pkmn_id,
  pkmn.Name AS pkmn_name,
  pkmn.Legendary AS pkmn_legendary
FROM 
  workspace.pokemons.pokemon pkmn
WHERE 
  pkmn.`Type 1` = "Grass"

UNION ALL

SELECT 
  pkmnd.`Pokedex Number` AS pkmn_id,
  pkmnd.Name AS pkmn_name,
  pkmnd.`Legendary Status` AS pkmn_legendary
FROM 
  workspace.pokemons.pokemon_details pkmnd
WHERE 
  pkmnd.Classification LIKE '%Bug%'

# HAVING

Usado para filtrar os resultados após o GROUP BY. É como um WHERE, mas aplicado em dados agregados.

`⚠️ WHERE filtra antes do agrupamento, HAVING filtra depois.`

In [0]:
-- Retorna os tipos de pokémons com mais de 50 ocorrências, ordenados pela quantidade decrescente
SELECT 
  pkmn.`Type 1` AS pkmn_type,
  COUNT(*) AS pkmn_qtd
FROM workspace.pokemons.pokemon pkmn
GROUP BY pkmn.`Type 1`
HAVING COUNT(*) > 50
ORDER BY pkmn_qtd DESC